# Lab 8: Experiments and Models

---

Welcome to lab 8! This week, we will be completing the hiring test that Enroll America used to screen their data analysts. Enroll America was a non-profit group that used data science to help sign people up for health insurance under the Affordable Care Act.

In [ ]:
# Before You Begin: Disable AI Assistance

To ensure you learn the concepts and complete this assignment based on your own understanding, you are *strongly* encouraged to turn off Google Colab's built-in generative AI features before you begin. This will also help you prepare for the midterms and final project.

**Follow these steps:**
1.  Go to the **Edit** menu at the top of the page.
2.  Click on **Notebook settings**.
3.  Check the box next to **"Hide generative AI features"**.
5.  Click **Save**.

This will prevent Google Gemini from suggesting or writing code for you, allowing you to focus on solving the problems yourself.



# Set-up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Bring in model data for Part 1
data_model = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/ea_test_model_analysis.csv")
data_model_dict = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/model_data_dict.csv")
# Bring in experiment data for Part 2
data_experiment = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/ea_test_experiment_analysis.csv")
data_experiment_dict = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/experiment_data_dict.csv")

## Part 1: Model

The above file `data_model` contains a dataset that was in part used to build a predictive model identifying an individual’s likelihood of being uninsured (`uninsured_score`). A data dictionary is included in `data_model_dict`. The column `q1_healthcare` contains the results to a survey question asking if the respondent has health insurance or not.

Using the dataset, produce an evaluation of the model that will help someone understand how the model works. Some ideas to consider include basic summary information about the model, model validation, cross tabs of interesting demographic groups and useful visualizations. Be sure to calculate and report the accuracy, precision, and recall of the model.

First, take a look at the dictionary and the data.

In [ ]:
# Here is data_model
data_model.head()

In [ ]:
# Set pandas display options to show full content
# Make sure nothing is being cut off for you
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

# Display the dictionary
data_model_dict

**Below, evaluate and interpret the model. You need a mix of both code and textual interpretation.** Be sure to calculate and report the accuracy, precision, and recall of the model.

You **should not** reverse engineer the model. Instead, think about precision/accuracy/recall as well as helpful visualizations, like from Lab 7. Apply and interpret those terms.

Throughout this lab, you are interested in whether or not people lack insurance. To q1_healthcare, I would consider responses 2 or 3 as not having insurance and response 1 as having insurance. You can drop the people who refuse this question.

## Part 2: Experiment

The above file `data_experiment` contains a dataset with the results of a randomized controlled experiment conducted by a non-profit organization working on health insurance enrollment. A data dictionary is included in `data_experiment_dict`.

For the experiment, the control group was suppressed from receiving any contact from the organization for a month. The treatment group was included in the regular program of the organization, which consists of two forms of contact. Most were called by a field organizer or volunteer and, if contacted, encouraged by phone to enroll in health insurance. Additionally, if an individual was subscribed to the email list they also received several emails a week encouraging them to enroll in health insurance.

The most important outcome is whether an individual said they currently have health insurance when surveyed by phone 2 weeks after the end of the experiment. The results to the follow-up survey are in columns Q1 through Q15. Using the dataset, test whether the outreach efforts had any causal effect on insurance status in the treatment group. Is the insurance rate greater in the treatment group than in the control group? Is this difference statistically significant? Remember to use permutation tests, like in Lab 6, Questions 5 and 6. (Hint! You should also review this chapter you read for Lab 6: https://inferentialthinking.com/chapters/12/1/AB_Testing.html. Note that this code uses the `datascience` library in Python, while we instead need to use `pandas` and `NumPy`. You will need to make some changes to the code from this textbook in order to get it to run for you.)

First, take a look at the dictionary and the data.

In [ ]:
# Here is data_experiment
pd.set_option('display.max_columns', 43) #https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook
data_experiment.head()

In [ ]:
# Here is the dictionary
data_experiment_dict

**Below, analyze the experiment. You need a mix of both code and textual interpretation.**

## Part 3

Based on your analysis from Part 2, briefly propose a follow-up randomized controlled experiment. The proposed test should be designed to expand upon the knowledge derived from the first test.

**Enter your proposal here.**

# Congratulations!

You are done with the lab. Before you finish and submit, please fill out this brief evaluation:

- I spent around XXXX hours on this lab,.
- This lab was (too easy, too hard, just about the right difficulty).

All assignments in the course will be distributed as notebooks like this one, and you will submit your work as a PDF.  

# How to Convert your Colab notebook to a PDF and download it

Follow these instructions exactly to make sure your notebook is correctly converted to a PDF and saved to your computer.

---

## 1. Check everything is in order and all the code runs

Before starting the conversion process, make sure your notebook is complete and error-free.

1. **Open your notebook** in Google Colab.
2. **Save your work**:
   - Go to **File → Save** or press `Ctrl + S` (Windows) / `Cmd + S` (Mac).
   - This ensures that all your recent changes are stored.
3. **Run all the cells** to confirm there are no errors:
   - In the top menu, select **Cell → Run All**.
   - Colab will execute every cell in order.
4. **Watch for errors**:
   - If you see any red error messages, fix them before proceeding.
   - A notebook with errors will **not** convert to PDF correctly.
5. Once all cells run without errors, you can proceed to the next step.

---

## 2. Make sure your notebook is saved in Google Drive and named correctly
Before converting, confirm that your notebook is stored in your Google Drive inside the `Colab Notebooks` folder.

1. Look at the top-left corner of the Colab page — you’ll see the notebook’s current name next to two yellow circle icons.
2. Rename the file by directly clicking on the name, type the new name, and press **Enter**.  
  - Please rename the notebook to LASTNAME_FIRSTNAME_LAB#.pdf. So for this lab, I would call it Alberto_Stefanelli_Lab8.ipynb.
  - The name must end with `.ipynb`
3. Ensure the notebook is in your Google Drive (not in Colab’s temporary session storage):  
   - In the menu, click **File → Locate in Drive**.  
   - This will open the folder in Drive where the notebook is stored.  
   - If it’s not in `My Drive/Colab Notebooks`, move it there for easier access.

---

## 3. Install the required tools

We wrote some code (see below) to automatically convert your Notebook to PDF. When you run the provided code cell, the first step will install some essential pieces of software (i.e., Pandoc and Latex) inside your Colab environment. There is no need to exactly understand what is happening

**Important:**
- This installation will take between 2 to 5 minutes.
- Do **not** close or refresh the Colab page while it runs.

---

## 4. Mount your Google Drive in Colab

After installing the requirements, the code will ask Colab to **mount** your Google Drive. You can use both your personal or Yale account. This is needed because the notebook you are converting must be saved in Drive before it can be converted to PDF.

You will see a pop-up with a **link**:
1. Click the link.
2. Sign in to your Google account (use the same account where your Colab notebook is saved).
3. If prompted, copy the long **authorization code** provided.
  - Paste that code into the input box in Colab and press **Enter**.
5. This will connect your Google Drive to Colab
6. If the link does not appear, make sure your browser is not blocking pop-ups.

---

## 5. Enter your notebook’s file name

The code will now ask to enter your notebook’s exact file name

1. Type the **full name** of your notebook, including the `.ipynb` ending.  
  - Example: `Alberto_Stefanelli_Lab8.ipynb.`
2. Make sure the name matches exactly, including capitalization and underscores.
3. Press **Enter**.

---

## 7. Convert the notebook to PDF and download it

The code will convert the notebook file to a pdf. After the PDF is created, your browser will show a download pop-up or automatically save the file to your Downloads folder. You can now open the PDF with any PDF reader. Once you have your PDF and made sure eveything is in order, you can then upload it to Canvas.

---

## 8. If you see an error:

Double-check that:

- All cells run without errors
- Mounted Google Drive without errors
- Saved notebook to Google Drive and not locally or on Github
- Entered correct notebook file name with `.ipynb`.
- Conversion completed without any errors.
- PDF downloaded to your computer (download and pop-ups are not blocked by your browser)


**Fallback:** If the PDF export method below fails (for example, due to LaTeX or pandoc errors), you can use https://convert.ploomber.io/ as a fallback option. However, I strongly suggest trying the methods below first and using this fallback only as a last resort.

**If you run into any issues, please reach out for help**



In [ ]:
# Install requirements
!apt-get -qq update
!apt-get install -y pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic

from google.colab import drive, files

# Mount Google Drive
drive.mount('/content/drive')

# Ask for the notebook name
notebook_name = input(
    "Enter your notebook’s exact file name,\n"
    "exactly as shown in the top-left corner of the Colab page (next to the two yellow circle icons): "
)

# Build paths
input_path = f"/content/drive/MyDrive/Colab Notebooks/{notebook_name}"
output_path = input_path.replace(".ipynb", ".pdf")

# Convert to PDF
!jupyter nbconvert --to pdf "{input_path}"

# Download the PDF
files.download(output_path)